# Gradient Descent

Gradient Descent is an algorithm that minimizes some objective $J(\theta)$ where $\theta$ is $\in \mathbb{R}^d$ by iteratively updating the vector $\theta$ by moving in the direction of steepest descent multiplied by some step size or learning rate.

In essence, 

$\>\>\>\>$1: Pick an initial guess $\theta_0$    
$\>\>\>\>$2: $\>\>$while $\theta_{k+1} - \theta_{k} \leq precision$:    
$\>\>\>\>$3: $\>\>\>\>\>\>\>\>$ $\theta_{k+1}$ = $\theta_{k} -\nabla J_\theta(\theta_k)$ * $\alpha$.

This is relevant to Machine Learning because it offers a numerical solution to estimates of parameters that do not have a closed form solution.

## Estimating Multinomial Logistic Regression Parameters with Gradient Descent 

#### Cost Function
However, one needs to determine the objective $J(\theta)$ before executing this algorithm. In the case of linear regression, the sum of squared residuals could be used. In this case, one can use the idea of MLE to minimize the log liklihood.

In the case where $K = 2$, the likelihood function was $\prod_{i=1}^{n} p(\textbf x_i)^{y_i} (1-p(\textbf x_i))^{1-y_i}$ 

In the case where $K > 2$,

For $k \in K$ where $K$ is the set of labeled classes and $p$ is the number of features      
Let $p_k(\textbf x)$ be the posterior probability $P(Y = k | X = \bf x)$ and the model be defined as 

$p_k(\textbf x) = \frac{e^{\beta_{0}^k + \textbf w_k\cdot \textbf x}}{\sum_{k=1}^{K} e^{\beta_{0}^k + \textbf w_k\cdot \textbf x}}$ for $k=1...K$

One can see how each class $1 \leq k \leq K$ has its own set of parameters $w_k = (\beta_1,...,\beta_p)$

We want the optimal set of parameters(weights): argmax $L(\beta$ $|$ $ \textbf x)$

$$\prod_{i=1}^{N}\prod_{k=1}^{K}P(Y = k | X = \textbf x_i)^{k_i} = \prod_{i=1}^{N}\prod_{k=1}^{K} p_k(\textbf x_i)^{k_i}$$

Minimizing the negative log-liklihood is the same as maximizing the log-liklihood. 

$$J(\textbf w) = -log\prod_{i=1}^{N}\prod_{k=1}^{K} p_k(\textbf x_i)^{k_i} = -\sum_{i=1}^{N}\sum_{k=1}^{K} {k_i} * log(\hat{y}_{ki})$$

where $k_i$ is 1 if $\textbf x_i$ is labeled as class k

Finally, we introduce the L2 norm to obtain a regularized objective function.

$$J(\textbf w) = -log\prod_{i=1}^{N}\prod_{k=1}^{K} p_k(\textbf x_i)^{k_i} = -\sum_{i=1}^{N}\sum_{k=1}^{K} {k_i} * log(\hat{y}_{ki}) + \alpha R(\beta)$$

where $R(\beta) = ||\beta||^2_2 = \sum^{k}_{i=1}\beta_i^2$ and $p_k(\textbf x_i)^{k_i} = \hat{y}_{ki}$


#### Gradient of Cost Function

TODO

In [4]:
import numpy as np

In [134]:
class BatchGradientDescent:
    def __init__(self):
        self.rate = .01
        self.precision = .001
        self.reg = .001
    
    # Pre: X - design matrix
    #      y - labeled classes
    #      w - mxn parameter matrix of current model
    # Post: a vector of errors for each label
    def determineCost(self, X, y, w):
        yhat = softmax(np.dot(X,w))
        
        # sum k sets of squared beta_i
        r_theta = self.reg * np.sum(w * w, axis = 0)
        # * operator will ensure a valid summation over all samples for each label because the labels have been one-hot encoded
        J = -np.sum((y * np.log(yhat)),axis = 0)+r_theta
        return J
    
    # Post: the gradient of J evaluated at w
    def gradient(self, X, y, w):
        yhat = softmax(np.dot(X,w))
        print(np.dot(X,yhat-y))
        return 0
    
    # Pre: z - x dot w of dimension (n*p)multiply(p*k)=>n*k
    # Post: Softmax score for each observation
    def softmax(self,z):
        return np.exp(z) / np.sum(np.exp(z), axis=1, keepdims=True)
    
    def gradientDescent(self):
        return 0

In [130]:
from sklearn.preprocessing import OneHotEncoder
class LogisticRegression:
    
    # Pre - data matrix X
    # Post - data matrix X with bias term concatenated as first column 
    def addBias(self,X):
        X = np.concatenate((np.ones((X.shape[0],1)),X),axis=1)
        return X
    
    def normalize(self,X):                                                     
        X = (X - np.min(X))/float(np.max(X) - np.min(X) )
        return X
    
    def oneHotEncode(self,y):
        enc = OneHotEncoder(handle_unknown='ignore')
        enc.fit(y)
        return enc.transform(y).toarray()
    
    def train(self,trainX,trainy):
        
        trainX = self.addBias(trainX)
        trainX = self.normalize(trainX)
        trainy = self.oneHotEncode(trainy)
        
        p,k = trainX.shape[1],trainy.shape[1]
        w = np.random.random([p*k]).reshape((p,k))
        
        GD = BatchGradientDescent()
        print(GD.determineCost(trainX,trainy,w))
        GD.gradient(trainX,trainy,w)

In [50]:
# loading the mnist dataset
from keras.datasets import mnist
from matplotlib import pyplot
# load dataset
(trainX, trainy), (testX, testy) = mnist.load_data()

trainX = trainX.reshape((trainX.shape[0],28*28))
trainy = trainy.reshape((trainX.shape[0],1))

testX = testX.reshape((testX.shape[0],28*28))
testy = testy.reshape((testX.shape[0],1))

# summarize loaded and reshaped dataset
print('Train: X=%s, y=%s' % (trainX.shape, trainy.shape))
print('Test: X=%s, y=%s' % (testX.shape, testy.shape))

Train: X=(60000, 784), y=(60000, 1)
Test: X=(10000, 784), y=(10000, 1)


In [135]:
LogisticRegression().train(trainX,trainy)

[26542.55932035 26413.09405568 14741.14381946 30530.23747941
 36951.18949817 21313.4603345  50821.70221678 24627.02794491
 21866.27045249 19360.46238479]


ValueError: shapes (60000,785) and (60000,10) not aligned: 785 (dim 1) != 60000 (dim 0)